In [1]:
from DataFetcher import DataFetcher
from USTs import USTs
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime

In [6]:
auctions = DataFetcher().fetch_auction_data()
prices = DataFetcher().fetch_historical_UST_data(date=datetime(year=2024, month=9, day=20))

In [7]:
ust_set = USTs(auction_data=auctions, price_data=prices)
ust_set = ust_set.get_current_UST_set(include_FRNs=False, include_TIPS=False)

Merged auction and price data successfully
No missing or excess data
All CUSIPs are identical between DataFrames


In [8]:
ust_set[ust_set['Cusip'] == '912810QT8']

,Cusip,Security type,Rate,Maturity date,Buy,Sell,End of day,security_term,issue_date
330,912810QT8,Bond,3.125,2041-11-15,88.703125,88.625,88.9375,30-Year,2011-11-15
